# 📊 Analyse des Résultats Overcooked

Ce notebook permet d'analyser les résultats d'évaluation des layouts Overcooked de manière interactive.

## 🔧 Configuration

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")
sns.set_palette("husl")

print("✅ Bibliothèques chargées")

## 📁 Chargement des Données

Modifiez les chemins vers vos fichiers de résultats :

In [ ]:
# Chemins vers les fichiers de résultats
files = {
    'Coopératif': 'layout_evaluation_coop.json',
    'Solo': 'layout_evaluation_solo.json',
    'GreedyStay': 'layout_evaluation_greedy_stay.json',
    'Solo_Detailed': 'layout_evaluation_solo_detailed.json'
}

# Charger les données disponibles
data = {}
for mode, filepath in files.items():
    if Path(filepath).exists():
        with open(filepath, 'r', encoding='utf-8') as f:
            data[mode] = json.load(f)
        print(f"✅ {mode}: {filepath}")
    else:
        print(f"⚠️ {mode}: {filepath} (fichier non trouvé)")

print(f"\n📊 {len(data)} fichiers chargés")

## 🎯 Résumé Général

In [ ]:
# Créer un DataFrame de résumé
summary_data = []

for mode, mode_data in data.items():
    results = mode_data.get('results', {})
    
    total_layouts = len(results)
    viable_layouts = sum(1 for layout in results.values() if layout.get('viable', False))
    completed_layouts = sum(1 for layout in results.values() if layout.get('completion_rate', 0) > 0)
    perfect_layouts = sum(1 for layout in results.values() if layout.get('completion_rate', 0) == 1.0)
    
    summary_data.append({
        'Mode': mode,
        'Total_Layouts': total_layouts,
        'Viable_Layouts': viable_layouts,
        'Completed_Layouts': completed_layouts,
        'Perfect_Layouts': perfect_layouts,
        'Viability_Rate': viable_layouts / max(total_layouts, 1),
        'Completion_Rate': completed_layouts / max(total_layouts, 1),
        'Perfect_Rate': perfect_layouts / max(total_layouts, 1)
    })

summary_df = pd.DataFrame(summary_data)
print("📊 RÉSUMÉ PAR MODE:")
print(summary_df.to_string(index=False, float_format='%.2f'))

## 📈 Visualisation des Taux de Réussite

In [ ]:
# Graphique des taux de réussite
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Taux de complétion
axes[0].bar(summary_df['Mode'], summary_df['Completion_Rate'], alpha=0.7)
axes[0].set_title('Taux de Complétion par Mode')
axes[0].set_ylabel('Taux de Complétion')
axes[0].set_ylim(0, 1)
for i, v in enumerate(summary_df['Completion_Rate']):
    axes[0].text(i, v + 0.02, f'{v:.1%}', ha='center')

# Taux de réussite parfaite
axes[1].bar(summary_df['Mode'], summary_df['Perfect_Rate'], alpha=0.7, color='green')
axes[1].set_title('Taux de Réussite Parfaite par Mode')
axes[1].set_ylabel('Taux de Réussite Parfaite')
axes[1].set_ylim(0, 1)
for i, v in enumerate(summary_df['Perfect_Rate']):
    axes[1].text(i, v + 0.02, f'{v:.1%}', ha='center')

plt.tight_layout()
plt.show()

## 🏗️ Analyse Détaillée par Layout

In [ ]:
# Créer un DataFrame détaillé par layout
layout_data = []

all_layouts = set()
for mode_data in data.values():
    all_layouts.update(mode_data.get('results', {}).keys())

for layout_name in sorted(all_layouts):
    for mode, mode_data in data.items():
        layout_info = mode_data.get('results', {}).get(layout_name, {})
        
        completion_rate = layout_info.get('completion_rate', 0)
        avg_steps = 0
        fastest_steps = 0
        
        if completion_rate > 0:
            cm = layout_info.get('completion_metrics', {})
            avg_steps = cm.get('average_completion_steps', 0)
            fastest_steps = cm.get('fastest_completion_steps', 0)
        
        layout_data.append({
            'Layout': layout_name,
            'Mode': mode,
            'Viable': layout_info.get('viable', False),
            'Completion_Rate': completion_rate,
            'Games_Played': layout_info.get('games_played', 0),
            'Games_Completed': layout_info.get('games_completed', 0),
            'Avg_Steps': avg_steps,
            'Fastest_Steps': fastest_steps,
            'Efficiency': max(0, (600 - avg_steps) / 600) if avg_steps > 0 else 0
        })

layout_df = pd.DataFrame(layout_data)

# Tableau pivot pour comparaison
pivot_completion = layout_df.pivot(index='Layout', columns='Mode', values='Completion_Rate')
pivot_steps = layout_df.pivot(index='Layout', columns='Mode', values='Avg_Steps')

print("📋 TAUX DE COMPLÉTION PAR LAYOUT:")
print(pivot_completion.fillna(0).to_string(float_format='%.2f'))

print("\n⏱️ STEPS MOYENS PAR LAYOUT (0 = échec):")
print(pivot_steps.fillna(0).to_string(float_format='%.0f'))

## 🎨 Heatmap de Performance

In [ ]:
# Heatmap des taux de complétion
plt.figure(figsize=(12, 8))
sns.heatmap(pivot_completion.fillna(0), annot=True, fmt='.2f', cmap='RdYlGn', 
            cbar_kws={'label': 'Taux de Complétion'})
plt.title('Heatmap des Taux de Complétion par Layout et Mode')
plt.ylabel('Layout')
plt.xlabel('Mode d\'Évaluation')
plt.tight_layout()
plt.show()

## ⚡ Analyse de l'Efficacité

In [ ]:
# Graphique des efficacités
completed_data = layout_df[layout_df['Completion_Rate'] > 0]

if not completed_data.empty:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Distribution des steps par mode
    for mode in completed_data['Mode'].unique():
        mode_data = completed_data[completed_data['Mode'] == mode]
        axes[0].hist(mode_data['Avg_Steps'], alpha=0.6, label=mode, bins=10)
    
    axes[0].set_title('Distribution des Steps Moyens')
    axes[0].set_xlabel('Steps Moyens')
    axes[0].set_ylabel('Nombre de Layouts')
    axes[0].legend()
    
    # Efficacité par mode
    efficiency_by_mode = completed_data.groupby('Mode')['Efficiency'].mean()
    axes[1].bar(efficiency_by_mode.index, efficiency_by_mode.values, alpha=0.7)
    axes[1].set_title('Efficacité Moyenne par Mode')
    axes[1].set_ylabel('Score d\'Efficacité')
    axes[1].set_ylim(0, 1)
    
    for i, v in enumerate(efficiency_by_mode.values):
        axes[1].text(i, v + 0.02, f'{v:.3f}', ha='center')
    
    plt.tight_layout()
    plt.show()
    
    print("🏆 TOP 5 LAYOUTS LES PLUS EFFICACES:")
    top_layouts = completed_data.nlargest(5, 'Efficiency')[['Layout', 'Mode', 'Avg_Steps', 'Efficiency']]
    print(top_layouts.to_string(index=False, float_format='%.3f'))
else:
    print("❌ Aucune donnée de complétion disponible")

## 🧠 Analyse Comportementale (si données détaillées disponibles)

In [ ]:
# Analyser les données comportementales détaillées
behavioral_data = None
behavioral_mode = None

for mode, mode_data in data.items():
    if mode_data.get('data_type') == 'detailed_with_individual_games':
        behavioral_data = mode_data
        behavioral_mode = mode
        break

if behavioral_data:
    print(f"🧠 Analyse comportementale pour le mode: {behavioral_mode}")
    
    event_types = [
        'tomato_pickup', 'onion_pickup', 'dish_pickup', 'soup_pickup',
        'soup_delivery', 'potting_onion', 'potting_tomato'
    ]
    
    behavioral_summary = []
    
    for layout_name, layout_data in behavioral_data['results'].items():
        if 'individual_games' not in layout_data:
            continue
            
        games = layout_data['individual_games']
        completed_games = [g for g in games if g.get('completed', False)]
        
        if not completed_games:
            continue
        
        # Calculer les moyennes d'événements
        event_totals = {event: [0, 0] for event in event_types}
        
        for game in completed_games:
            info_sum = game.get('info_sum', {})
            for event in event_types:
                if event in info_sum:
                    event_totals[event][0] += info_sum[event][0]
                    event_totals[event][1] += info_sum[event][1]
        
        num_games = len(completed_games)
        
        # Ajouter aux données de résumé
        summary_row = {'Layout': layout_name, 'Completed_Games': num_games}
        
        for event in event_types:
            total_avg = sum(event_totals[event]) / num_games
            agent0_avg = event_totals[event][0] / num_games
            agent1_avg = event_totals[event][1] / num_games
            
            summary_row[f'{event}_total'] = total_avg
            summary_row[f'{event}_agent0'] = agent0_avg
            summary_row[f'{event}_agent1'] = agent1_avg
            
            # Balance entre agents
            if total_avg > 0:
                balance = min(agent0_avg, agent1_avg) / max(agent0_avg, agent1_avg) if max(agent0_avg, agent1_avg) > 0 else 1
                summary_row[f'{event}_balance'] = balance
            else:
                summary_row[f'{event}_balance'] = 1
        
        behavioral_summary.append(summary_row)
    
    if behavioral_summary:
        behavioral_df = pd.DataFrame(behavioral_summary)
        
        # Graphique des événements principaux
        main_events = ['soup_delivery', 'onion_pickup', 'tomato_pickup', 'dish_pickup']
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        axes = axes.flatten()
        
        for i, event in enumerate(main_events[:4]):
            if f'{event}_total' in behavioral_df.columns:
                axes[i].bar(behavioral_df['Layout'], behavioral_df[f'{event}_total'])
                axes[i].set_title(f'{event.replace("_", " ").title()} par Layout')
                axes[i].set_ylabel('Moyenne par partie')
                axes[i].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # Balance des agents pour les livraisons
        if 'soup_delivery_balance' in behavioral_df.columns:
            plt.figure(figsize=(12, 6))
            bars = plt.bar(behavioral_df['Layout'], behavioral_df['soup_delivery_balance'])
            
            # Colorier selon la balance
            for bar, balance in zip(bars, behavioral_df['soup_delivery_balance']):
                if balance > 0.7:
                    bar.set_color('green')  # Équilibré
                elif balance > 0.4:
                    bar.set_color('orange')  # Moyennement équilibré
                else:
                    bar.set_color('red')  # Spécialisé
            
            plt.title('Balance des Livraisons entre Agents')
            plt.ylabel('Score de Balance (1.0 = parfaitement équilibré)')
            plt.xlabel('Layout')
            plt.xticks(rotation=45)
            plt.axhline(y=0.7, color='green', linestyle='--', alpha=0.5, label='Seuil équilibré')
            plt.axhline(y=0.4, color='orange', linestyle='--', alpha=0.5, label='Seuil spécialisé')
            plt.legend()
            plt.tight_layout()
            plt.show()
        
        print("\n📊 RÉSUMÉ COMPORTEMENTAL:")
        key_cols = ['Layout', 'Completed_Games', 'soup_delivery_total', 'soup_delivery_balance']
        if all(col in behavioral_df.columns for col in key_cols):
            print(behavioral_df[key_cols].to_string(index=False, float_format='%.2f'))
else:
    print("⚠️ Aucune donnée comportementale détaillée disponible")
    print("   Utilisez un fichier avec data_type: 'detailed_with_individual_games'")

## 💾 Exporter les Résultats

In [ ]:
# Exporter les DataFrames vers CSV
output_dir = Path('analysis_output')
output_dir.mkdir(exist_ok=True)

# Résumé général
summary_df.to_csv(output_dir / 'summary_by_mode.csv', index=False)
print(f"✅ Résumé exporté: {output_dir / 'summary_by_mode.csv'}")

# Données détaillées par layout
layout_df.to_csv(output_dir / 'detailed_by_layout.csv', index=False)
print(f"✅ Détails exportés: {output_dir / 'detailed_by_layout.csv'}")

# Tableaux pivot
pivot_completion.to_csv(output_dir / 'completion_rates_matrix.csv')
pivot_steps.to_csv(output_dir / 'average_steps_matrix.csv')
print(f"✅ Matrices exportées: {output_dir / 'completion_rates_matrix.csv'} et {output_dir / 'average_steps_matrix.csv'}")

# Données comportementales si disponibles
if 'behavioral_df' in locals():
    behavioral_df.to_csv(output_dir / 'behavioral_analysis.csv', index=False)
    print(f"✅ Analyse comportementale exportée: {output_dir / 'behavioral_analysis.csv'}")

print(f"\n📁 Tous les fichiers exportés dans: {output_dir.absolute()}")

## 🎯 Conclusions et Recommandations

In [ ]:
print("🎯 CONCLUSIONS ET RECOMMANDATIONS")
print("=" * 40)

# Analyser les performances par mode
if len(data) > 1:
    print("\n📊 COMPARAISON DES MODES:")
    
    best_completion = summary_df.loc[summary_df['Completion_Rate'].idxmax()]
    worst_completion = summary_df.loc[summary_df['Completion_Rate'].idxmin()]
    
    print(f"• Meilleur mode (complétion): {best_completion['Mode']} ({best_completion['Completion_Rate']:.1%})")
    print(f"• Mode le moins performant: {worst_completion['Mode']} ({worst_completion['Completion_Rate']:.1%})")
    
    # Identifier les layouts problématiques
    problematic_layouts = []
    reliable_layouts = []
    
    for layout in pivot_completion.index:
        completion_rates = pivot_completion.loc[layout].dropna()
        if completion_rates.max() == 0:
            problematic_layouts.append(layout)
        elif completion_rates.min() >= 0.8:
            reliable_layouts.append(layout)
    
    if problematic_layouts:
        print(f"\n❌ Layouts problématiques (échec dans tous les modes): {', '.join(problematic_layouts)}")
    
    if reliable_layouts:
        print(f"\n✅ Layouts fiables (>80% dans tous les modes): {', '.join(reliable_layouts)}")

# Recommandations d'amélioration
print("\n🔧 RECOMMANDATIONS:")

# Analyser les écarts de performance
if 'Coopératif' in data and 'Solo' in data:
    coop_completion = summary_df[summary_df['Mode'] == 'Coopératif']['Completion_Rate'].iloc[0]
    solo_completion = summary_df[summary_df['Mode'] == 'Solo']['Completion_Rate'].iloc[0]
    
    if solo_completion > coop_completion + 0.2:
        print("• Le mode Solo surpasse significativement le mode Coopératif")
        print("  → Améliorer les mécanismes de coordination entre GreedyAgents")
        print("  → Considérer des stratégies de communication ou de planification partagée")
    
    elif coop_completion > solo_completion + 0.2:
        print("• Le mode Coopératif surpasse le mode Solo")
        print("  → Les layouts bénéficient de la coopération")
        print("  → Continuer le développement de stratégies collaboratives")

# Recommandations basées sur l'efficacité
if not completed_data.empty:
    avg_efficiency = completed_data['Efficiency'].mean()
    if avg_efficiency < 0.3:
        print("• Efficacité générale faible détectée")
        print("  → Optimiser les algorithmes de pathfinding")
        print("  → Revoir la logique de prise de décision des agents")
    
    elif avg_efficiency > 0.7:
        print("• Excellente efficacité générale")
        print("  → Layouts bien adaptés aux agents actuels")
        print("  → Considérer des défis plus complexes")

print("\n✨ Analyse terminée! Utilisez les fichiers CSV exportés pour des analyses plus poussées.")